In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [5]:
# Comenzaremos creando una función que, dada una latitud y longitud, calcule la distancia a otro punto de latitud y longitud conocidos.
import math

def distancia(lat1, lon1, lat2, lon2):
    # Radio de la Tierra en kilómetros
    R = 6371.0
    
    # Convertir grados a radianes
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Diferencias entre las coordenadas
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Fórmula de Haversine
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Distancia en kilómetros
    distance = R * c
    return distance

In [113]:
# Cargamos los datos
df = pd.read_csv(r'C:\Users\usuario\Desktop\Universidad\TERCERO\Primer_Cuatri\Hackaton\todo_junto.csv')
df.head()

C:\Users\usuario\AppData\Local\Temp\ipykernel_18388\2260218128.py:2: DtypeWarning: Columns (12,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\usuario\Desktop\Universidad\TERCERO\Primer_Cuatri\Hackaton\todo_junto.csv')


,ID,AMBITO_NOMBRE,EMBALSE_NOMBRE,AGUA_TOTAL,ELECTRICO_FLAG,CODIGO,EMBALSE,X,Y,DEMARC,...,OPENSTREETMAP,WIKIDATA,PROVINCIA,CCAA,TIPO,COTA_CORON,ALT_CIMIEN,INFORME,FECHA,AGUA_ACTUAL
0,170,GUADIANA,BOQUERÓN,6.0,0.0,4060008.0,BOQUERON,"39,1511076260001","-6,41827824299997",GUADIANA,...,NaN,NaN,Badajoz,Extremadura,Presa de fábrica de gravedad (hormigón vibrado),NaN,NaN,https://sig.mapama.gob.es/WebServices/clientew...,01/11/16 00:00,4.0
1,170,GUADIANA,BOQUERÓN,6.0,0.0,4060008.0,BOQUERON,"39,1511076260001","-6,41827824299997",GUADIANA,...,NaN,NaN,Badajoz,Extremadura,Presa de fábrica de gravedad (hormigón vibrado),NaN,NaN,https://sig.mapama.gob.es/WebServices/clientew...,24/01/17 00:00,4.0
2,170,GUADIANA,BOQUERÓN,6.0,0.0,4060008.0,BOQUERON,"39,1511076260001","-6,41827824299997",GUADIANA,...,NaN,NaN,Badajoz,Extremadura,Presa de fábrica de gravedad (hormigón vibrado),NaN,NaN,https://sig.mapama.gob.es/WebServices/clientew...,28/02/17 00:00,5.0
3,170,GUADIANA,BOQUERÓN,6.0,0.0,4060008.0,BOQUERON,"39,1511076260001","-6,41827824299997",GUADIANA,...,NaN,NaN,Badajoz,Extremadura,Presa de fábrica de gravedad (hormigón vibrado),NaN,NaN,https://sig.mapama.gob.es/WebServices/clientew...,21/03/17 00:00,5.0
4,170,GUADIANA,BOQUERÓN,6.0,0.0,4060008.0,BOQUERON,"39,1511076260001","-6,41827824299997",GUADIANA,...,NaN,NaN,Badajoz,Extremadura,Presa de fábrica de gravedad (hormigón vibrado),NaN,NaN,https://sig.mapama.gob.es/WebServices/clientew...,02/05/17 00:00,5.0


In [11]:
# Vamos a obtener todos los embalses que tenemos en nuestro dataframe
id_embalses = df['ID'].unique()
print(f'Hay {len(id_embalses)} embalses en el dataframe.')

Hay 248 embalses en el dataframe.


In [12]:
# Análisis estadístico de los datos que tenemos
df.describe()

,ID,AGUA_TOTAL,ELECTRICO_FLAG,CODIGO,AGUA_ACTUAL
count,494422.000000,493475.000000,494422.000000,4.944220e+05,494420.000000
mean,373.539151,165.231953,0.217812,5.156877e+06,84.658108
std,212.438022,304.589871,0.412759,2.989339e+06,173.812860
min,3.000000,5.000000,0.000000,1.090002e+06,0.000000
25%,186.000000,20.000000,0.000000,3.100057e+06,9.000000
50%,384.000000,53.000000,0.000000,4.210052e+06,26.000000
75%,562.000000,152.000000,0.000000,8.030035e+06,81.000000
max,741.000000,2649.000000,1.000000,1.520001e+07,2564.000000


In [22]:
# Obtenemos la relación de cada embalse con sus coordenadas
listado = df[['ID', 'EMBALSE_NOMBRE', 'X', 'Y']].drop_duplicates()
listado.head()

,ID,EMBALSE_NOMBRE,X,Y
0,170,BOQUERÓN,"39,1511076260001","-6,41827824299997"
1917,462,BÚBAL,"42,68098159","-0,315761256999963"
3834,255,BUENDÍA,"40,3992227420001","-2,78382182099995"
5751,725,CASTREJÓN,"39,833565075","-4,29494820899993"
7668,571,CELEMÍN,"36,299322019","-5,77443215499994"


In [48]:
# Vamos a conertir X e Y a tipo float32
listado['X'] = listado['X'].replace(',', '.', regex=True)
listado['Y'] = listado['Y'].replace(',', '.', regex=True)
listado['X'] = listado['X'].astype(np.float32)
listado['Y'] = listado['Y'].astype(np.float32)

In [94]:
'''
Aunque no se ha llegado a conectar con un backend para que se comunique con el fronted, aquí hemos llegado a
implementar una función que, dada una latitud y longitud, nos devuelva los embalses que se encuentran a una distancia
menor a 100 km de ese punto, ordenadas por distancia.
'''

def buscar_embalses(X, Y):
    cercanos = {}
    for id in id_embalses:
        lat = listado.loc[listado['ID'] == id, 'X'].values[0]
        lon = listado.loc[listado['ID'] == id, 'Y'].values[0]
        for embalse in id_embalses:
            distancia_km = distancia(lat, lon, X, Y)
            if distancia_km <= 100:
                cercanos[listado.loc[listado['ID'] == id, 'EMBALSE_NOMBRE'].values[0]] = distancia_km
    cercanos = dict(sorted(cercanos.items(), key=lambda item: item[1]))
    return cercanos

In [95]:
buscar_embalses(40.4165, -3.70256)

{'PARDO, EL': 15.43482991976734,
 'VALMAYOR': 32.013315799177306,
 'NAVACERRADA': 41.885067358094695,
 'JAROSA, LA': 44.644579545998376,
 'PICADAS': 47.218561698847,
 'SAN JUAN': 52.381238949731475,
 'ATAZAR, EL': 58.38247762474611,
 'VILLAR, EL': 60.20118053910352,
 'RIOSEQUILLO': 63.335510637835576,
 'ALMOGUERA': 64.9829630448638,
 'PUENTES VIEJAS': 65.06611060727822,
 'CASTRO, EL': 67.97011245211786,
 'BELEÑA': 71.65433717748628,
 'SERONES': 71.90866159245442,
 'VADO, EL': 73.52785144842474,
 'BOLARQUE': 75.09091982741539,
 'GUAJARAZ': 75.9763187385106,
 'BUENDÍA': 77.81229170672695,
 'ENTREPEÑAS': 80.97540805221892,
 'CASTREJÓN': 82.08740600426626,
 'FINISTERRE': 85.40928436404705,
 'ALCORLO': 87.16518214637358,
 'CASTRO DE LAS COGOTAS': 90.72387718887147,
 'PÁLMACES': 95.38504377844225,
 'CAZALEGAS': 96.31370297904533}

In [98]:
# Haremos una función, dado un id de un embalse, devuelva todas las fechas y cantidades de agua embalsada
def embalse_info(id):
    return df.loc[df['ID'] == id, ['FECHA', 'AGUA_TOTAL']]